In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH


In [2]:
from textwrap import wrap
import os
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
datasetPathDatabaseAdditional =  configParser.get('COMMON', 'datasetPathDatabase') + '/datasetAdditional.db'
datasetPathDatabaseVgg =  configParser.get('COMMON', 'datasetPathDatabase') + '/datasetFacesBlurred.db'
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
unet_dim =  int(configParser.get('COMMON', 'unet_dim'))
unet1_image_size =  int(configParser.get('COMMON', 'unet1_image_size'))
audio_embs =  str(configParser.get('COMMON', 'audio_embs'))
audio_length_used =  configParser.get('train_imagen', 'audio_length_used') 
model_filename =  configParser.get('train_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
sub_epochs=  int(configParser.get('train_imagen', 'sub_epochs') )
batch_size=  int(configParser.get('train_imagen', 'batch_size') )
sample_every=  int(configParser.get('train_imagen', 'sample_every') ) - 1
save_model_every=  int(configParser.get('train_imagen', 'save_model_every') ) - 1
epochs=  int(configParser.get('train_imagen', 'epochs') )
sample_every_offset=  int(configParser.get('train_imagen', 'sample_every_offset') ) - 1
save_every_offset=  int(configParser.get('train_imagen', 'save_every_offset') ) - 1

continue_from_epoch = int(configParser.get('train_imagen', 'continue_from_epoch'))
continue_from_offset = int(configParser.get('train_imagen', 'continue_from_offset'))
continue_from_epoch_and_offset_flag = int(configParser.get('train_imagen', 'continue_from_epoch_and_offset_flag'))
db_chunk = int(configParser.get('train_imagen', 'db_chunk'))


pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None



In [3]:
def speaker_emb_preprocess(speaker_emb2):

    speaker_emb2 = pickle.loads(speaker_emb2)
    speaker_emb2 = speaker_emb2.squeeze()
    speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
    speaker_emb2 = np.tile(speaker_emb2, (1, 1))
    speaker_emb2 = speaker_emb2 / 200.0
    speaker_emb2 = np.array(speaker_emb2).tolist()
    return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    speaker_emb2 = pickle.loads(speaker_emb2)
    a = speaker_emb2 #np.zeros(shape=(24, 512))
    a = a[-39:,:] # Actually only last 39 are the relevant ones
    #print('au')
    #print(a.shape)
    b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
    c = np.zeros(shape=(40-a.shape[0], 768))
    arr = np.concatenate((a, b), axis=1)
    arr = np.concatenate((arr, c), axis=0)
    arr = arr / (1 if(audio_embs == 'wav2vec') else 10 if(audio_embs == 'openl3')  else 1 if(audio_embs == 'pyannoteTitaNet') else 1)
    #print(str(arr.max()))
    speaker_emb2 = np.array(arr).tolist()
    return speaker_emb2

def audio_features_preprocess(video_id):
    #print(video_id)
    conAdditional = sl.connect(datasetPathDatabaseAdditional)
    cur = conAdditional.cursor()
    sql = '''SELECT AUDIO_FEATURES FROM AUDIO WHERE VIDEO_ID = ?'''
    cur.execute(sql, [video_id])
    audio_features = cur.fetchall()
    #print(audio_features[0])
    audio_features = pickle.loads(audio_features[0][0])
    #print(audio_features.shape) # 190 x 128
    audio_features = audio_features[0:161]
    z1 = np.zeros((161,768-128))
    audio_features = np.hstack((audio_features,z1))
    audio_features = audio_features / 100.0
    return audio_features

def audio_transformer_features_preprocess(video_id):
    #print(video_id)
    conAdditional = sl.connect(datasetPathDatabaseAdditional)
    cur = conAdditional.cursor()
    sql = '''SELECT AUDIO_FEATURES FROM AUDIO_TRANS WHERE VIDEO_ID = ?'''
    cur.execute(sql, [video_id])
    audio_features = cur.fetchall()
    #print(audio_features[0])
    audio_features = pickle.loads(audio_features[0][0])
    #print(audio_features.shape) # 1214 x 768
    audio_features = audio_features.squeeze()
    audio_features = audio_features[0:161]
    #z1 = np.zeros((161,768-128))
    #audio_features = np.hstack((audio_features,z1))
    #audio_features = audio_features / 100.0
    return audio_features

import random
def image_guide_preprocess(face_id):
    #print(face_id)

    if(random.random() < 0.5):
        image_guide = np.zeros((49,768))
        #print(image_guide.shape)
    else:
        conAdditional = sl.connect(datasetPathDatabaseVgg)
        cur = conAdditional.cursor()
        sql = '''SELECT BLURRED_FACE_EMB FROM FACES_BLURRED WHERE FACE_ID = ?'''
        cur.execute(sql, [face_id])
        image_guide = cur.fetchall()
        #print(image_guide)
        image_guide = pickle.loads(image_guide[0][0])
        image_guide = image_guide.squeeze() #197 x 768
        image_guide = image_guide[1::4] #49 x 768
    return image_guide



In [4]:



def process_age(age):
    if(random.random() < 0.2):
        x = np.zeros(768)
        x[767] = 1
        return x
    else:
        x = np.ones(768) * (age / 100.0)
        x[767] = 0
        return x

def process_gender(gender):
    if(random.random() < 0.2):
        return np.zeros(768)
    elif(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(random.random() < 0.2):
        x = x
    elif(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(random.random() < 0.2):
        x = x
    elif(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x       

In [5]:

def process_input(age,ethnicity,gender,language,speaker_emb,audio_emb,audio_features,image_guide):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((age, ethnicity))
    h = np.vstack((h, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))
    h = np.vstack((h,audio_features))
    h = np.vstack((h,image_guide))
    #print('aaaaaaaaaaa')
    #print(h.shape)
    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()

from PIL import Image
def make_square(im, min_size=image_size, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    im.close()
    return new_im

def process_image_path(path):
    
    image = Image.open(path).convert('RGB')

    w_s = image_size / (1+2 * 0.5)
    h_s = image_size / (1+2 * 0.4)

    image = image.crop((0.4*w_s,0.0*h_s,1.6*w_s,1.4*h_s))

    image = make_square(image)
    #print('saving')
    #image.save(str(random.random()) + '.png')

    

    #print(np.array(image,np.float32).shape)
    pix = np.array(image,np.float32)
    pix = np.moveaxis(pix, -1, 0)
    
    pix = pix / 255
    image.close()
    return pix.tolist()

In [6]:
import multiprocessing
import train_imagen_batch

procs = []

range_of_epochs = range(1,epochs + 1)

if(continue_from_epoch_and_offset_flag != 0):
    
    range_of_epochs = range(continue_from_epoch, epochs + 1)

offset_zero_flag = 0

for epoch in range_of_epochs:

    print("Training in epoch " + str(epoch))

    offset = 0

    if(continue_from_epoch_and_offset_flag != 0 and offset_zero_flag == 0):
        offset = continue_from_offset
        offset_zero_flag = 1

    while(True):
        con = sl.connect(datasetPathDatabase)
        print("Training in offset " + str(offset))

        print("Getting Data...")
        ## TODO: NEED TO CHECK F.ID CORRESPONDS TO CORRECT FACE_PATH

        data = con.execute("SELECT V.ID,F.ID,V.VIDEO_PATH, V.AGE CAPTION_A, " + 
                            "V.ETHNICITY CAPTION_E, " +
                            "lower(V.GENDER) CAPTION_G, " +
                            "A.SPEAKER_EMB, "+ ("A.WAV_TO_VEC, " if(audio_embs == 'wav2vec') else "A.AUDIO_EMB2, " if(audio_embs == 'openl3')  else "A.PYANNOTE_TITANET, " if(audio_embs == 'pyannoteTitaNet') else ', ') +
                            "A.AUDIO_FEATURES, " +
                            "A.LANG CAPTION_L, "+
                            "F.FACE_PATH "+
                            "FROM VIDEO V "+
                            "INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID " +
                            "INNER JOIN FACE F ON F.ID = (select ID from FACE f2 where f2.video_id = v.ID ORDER By ID limit 1 ) " + 
                            "WHERE AUDIO_PRE = 4 AND FACES_PRE = 2 AND AUDIO_LENGTH = " + audio_length_used + ' ' +
                            "AND V.ID in (select V2.ID from VIDEO v2 order by V2.ID LIMIT "+ str(db_chunk) +" OFFSET " + str(offset) + ")")
        dataGotten = data.fetchall()

        
        print("Data Gotten")

        if(len(dataGotten) == 0):
            break

        

        print("Preprocessing Data...")
        df = pd.DataFrame(dataGotten,columns = ['ID','FACE_ID','VIDEO_PATH','caption_a','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','AUDIO_FEATURES','caption_l','image_path'])
        df["image_guide"] = np.nan


        #print(df.head(10))
        data_frame = df[["ID","FACE_ID","image_path","caption_a","caption_e","caption_g","caption_l"]]
        data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
        data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
        data_frame['image_guide'] = df['image_guide']

        
        data_frame['SPEAKER_EMB'] = data_frame['SPEAKER_EMB'].apply(lambda x: speaker_emb_preprocess(x))
        data_frame['AUDIO_EMB'] = data_frame['AUDIO_EMB'].apply(lambda x: audio_emb_preprocess2(x))
        data_frame['AUDIO_FEATURES'] = data_frame['ID'].apply(lambda x: audio_transformer_features_preprocess(x))
        data_frame = data_frame.drop(['ID'], axis=1)
        data_frame['image_guide'] = data_frame['FACE_ID'].apply(lambda x: image_guide_preprocess(x))
        data_frame = data_frame.drop(['FACE_ID'], axis=1)
        data_frame['caption_a'] = data_frame['caption_a'].apply(lambda x: process_age(x))
        data_frame['caption_g'] = data_frame['caption_g'].apply(lambda x: process_gender(x))
        data_frame['caption_l'] = data_frame['caption_l'].apply(lambda x: process_language(x))
        data_frame['caption_e'] = data_frame['caption_e'].apply(lambda x: process_ethnicity(x))

        data_frame['INPUT'] = data_frame['SPEAKER_EMB']

        for index, row in data_frame.iterrows():
            x = process_input(data_frame.loc[index,"caption_a"],data_frame.loc[index,"caption_e"]
                              ,data_frame.loc[index,"caption_g"],
                            data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                            data_frame.loc[index,"AUDIO_EMB"],data_frame.loc[index,"AUDIO_FEATURES"],
                            data_frame.loc[index,"image_guide"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"INPUT"] = x

        data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)


        data_frame['image_path'] = data_frame['image_path'].apply(lambda x: process_image_path(x))

        input = data_frame['INPUT'].to_numpy()
        input = np.array([np.array(xi) for xi in input])
        input[np.isnan(input)] = 0
        input[input > 10] = 10
        input[input < -10] = -10

        
        output = data_frame['image_path'].to_numpy()
        output = np.array([np.array(xi) for xi in output])
        output.squeeze().shape

        print("Data Preprocessed...")

        
        #print(len(procs))
        #if(len(procs) > 0):
        #    procs[0].join() # Wait for previous process to finish
        #    print("Model trained using a batch of data...")
        #    procs = []
        #print(len(procs))

        print("Training model using another batch of data...")
        proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet1, args=(input,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,unet_dim,unet1_image_size,))
        #procs.append(proc)
        proc.start()
        proc.join()

        #print("Model trained using this batch of data...")

        #proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet2, args=(input,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,unet1_dim,unet2_dim,))
        #proc.start()two_unets_pyannote_nemo
        #proc.join()
        offset = offset + db_chunk

        
        data.close()
        con.close()
        del dataGotten
        del df
        del data_frame
        del input
        del output
        gc.collect()

Training in epoch 0
Training in offset 500
Getting Data...
Data Gotten
Preprocessing Data...
751
754
757
760
763
88423
88426
88429
88432
766
769
772
775
778
168143
88435
781
784
787
790
793
88438
88441
88444
88447
168146
796
799
802
805
808
168149
168152
168155
88450
811
814
817
820
823
88453
88456
88459
88462
826
829
832
835
838
168158
88465
841
844
847
850
853
88468
88471
88473
88476
168161
856
859
862
865
868
168164
168167
168170
88479
871
874
877
880
883
88482
88485
88488
88491
886
889
892
895
898
168173
88494
901
904
907
910
913
88497
88500
88503
88506
168176
916
919
922
925
928
168179
168182
168185
88509
931
934
937
940
943
88512
88515
88518
88521
946
949
952
955
958
168188
88524
961
964
967
970
973
88527
88530
88533
88536
168191
976
979
982
985
988
168194
168197
168200
88539
991
994
997
1000
1003
88542
88545
88548


KeyboardInterrupt: 